In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

import spacy
from spacy import displacy

def display_ent(doc, style="ent", colors=None, options=None, compact=True):
    colors = colors or {"TRIGGER": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
    options = options or {"ents": None, "colors": colors, "compact": compact}
    displacy.render(doc, style=style, jupyter=True, options=options)

from pipeline.utils.display import display_ent
from pipeline.data.metadata import get_report_data, get_geoview_data
from pipeline.preprocessing.processing import match_triggers, triggers_json_to_df, load_triggers, load_json

classifier.py
helpers.py
processing.py
metadata.py


In [3]:
# testing engineering for extracting triggers from entity ruler, rather than on matcher
from pipeline.preprocessing.processing import load_spacy_model, create_trigger_ruler

nlp = load_spacy_model(output_type='doc', trigger_matcher=True, lemmatizer=False, geological_matcher=True,
                       stopword_removal=False, punctuation_removal=True, lemmatize_triggers=True)

Added punctuation removal pipe
Added geological entity matcher pipe
Added trigger phrase matcher pipe
Loading spaCy model with spaCy doc output.


# Load DataFrame

In [4]:
capstone_files, files = get_report_data(count_sentences=True, return_files=True)

Loading files as dict: 100%|██████████| 32646/32646 [00:02<00:00, 13712.93it/s]


In [5]:
# full sweep of all documents on triggers to build data manually
# data = {file : [(idx, ent.text) for idx, doc in enumerate(nlp.pipe(
#     sentences, disable=['ner'], n_process=1, cleanup=True, batch_size=10000))
#     for ent in doc.ents if ent.label_ == 'TRIGGER'] for file, sentences in tqdm(
#     files.items(), desc='Extracting TRIGGER entities from documents')}

# from pipeline.preprocessing.processing import data_path
# import json

# with open(data_path / 'sentence_triggers.json', 'w+') as f:
#     json.dump(data, f)

# df = pd.DataFrame([
#     {"filename": filename, "idx": int(match[0]), "trigger": match[1]} 
#     for filename, matches in data.items() for match in matches
# ])

# df.groupby(['filename','idx'])['trigger'].apply(
#     lambda x : ', '.join(x).strip()).reset_index().rename(
#     columns={'trigger':'triggers'})#.to_csv('extracted_triggers_grouped.csv', index_label='triggers_id')


In [6]:
# specify labellers
users = ('daniel','charlie')
dataset = {}

### SPECIFY USER ###
user = 'daniel'

# get data from path given user
from pathlib import Path
base_path = Path('.')
labelled_path = base_path / 'data' / 'events' / f'{user}_dataset.csv'

# get geoview data
geoview = get_geoview_data()

# load triggers grouped by sentence_idx in each file - triggers are comma separated strings
extracted_triggers = pd.read_csv(os.path.join('data','events', 'extracted_triggers_grouped.csv'), index_col=0)

# merge capstone_file data with anumber data with extracted triggers, joined on filename
source = capstone_files.merge(
    geoview[['anumber','report_type']], on='anumber').merge(
    extracted_triggers, on='filename')

for usr in users:
    if usr==user:
        if os.path.isfile(labelled_path):
            print(f'File for {user} at {labelled_path.resolve()} already exists. Loading file.')
            dataset[user] = pd.read_csv(labelled_path, index_col=0)
        else:
            print(f'Creating new dataset for {user}.')
            # create a dataset for each labeller based off the source data
            dataset[user] = pd.DataFrame.from_dict(dict(
                    source.to_dict(),
                    **{'reviewed': {idx: False for idx in range(len(source))}},
                    **{'label': {idx: False for idx in range(len(source))}},
                    **{'confidence': {idx: None for idx in range(len(source))}},
                    **{'lower_bound': {idx: 0 for idx in range(len(source))}},
                    **{'upper_bound': {idx: 0 for idx in range(len(source))}}))

dataset[user].head()

File for daniel at /home/daniel/capstone/data/events/daniel_dataset.csv already exists. Loading file.


,filename,anumber,sentence_count,report_type,idx,triggers,reviewed,label,confidence,lower_bound,upper_bound
index,,,,,,,,,,,
0,a074282_reidy_annual2006_12863956.json,74282,141,Annual,19,prospect,False,False,NaN,0,0
1,a074282_reidy_annual2006_12863956.json,74282,141,Annual,43,"anomalism, mineralization",False,False,NaN,0,0
2,a074282_reidy_annual2006_12863956.json,74282,141,Annual,53,"supported, mineralization",False,False,NaN,0,0
3,a074282_reidy_annual2006_12863956.json,74282,141,Annual,84,extensive,False,False,NaN,0,0
4,a074282_reidy_annual2006_12863956.json,74282,141,Annual,123,presence of,False,False,NaN,0,0


In [7]:
dataset[user].to_csv(labelled_path, index=True, index_label='index')

In [8]:
from sklearn.model_selection import train_test_split

seed = 10

def stratify_source_data(source, n, prop=False, seed=None, stratify_col='report_type'):
    if prop:  # if n is not a percentage/proportion
        n /= len(source)  # calculate count of total
    train_idx, test_idx, train_label, test_label = train_test_split(source.index, source.label,
        test_size = n, random_state = seed, stratify = source[stratify_col])
    
    return train_idx, test_idx, train_label, test_label

def get_indices(dataset, n=100, prop=False, seed=None):
    _, indices, _, _ = stratify_source_data(dataset, n=n, prop=prop, seed=seed)
    return indices

In [9]:
# get indices from stratified sampling procedure - make sure to set seed for consistent
batch_size = 100
batch_indices = get_indices(dataset[user], n=batch_size, seed=seed)
batch_indices

Int64Index([285173,  31753,  30227, 286567, 416405, 348130,  34561, 250226,
            328562, 260488, 567911,  69616, 544614, 574056, 224878, 227250,
            274951, 128062,  67388, 265521, 399504, 418206, 371608, 420438,
            580573, 445860, 576365, 305482, 366571, 418992, 125671, 254945,
            252887, 473485,  75026, 494942, 273298, 389623, 428408, 234677,
            186454,   9522, 317052, 474140, 489705,  27937,  12256,  90785,
            396561, 233878, 457919,  36048, 373581, 351670, 562448, 432349,
            296739, 122783, 438533,  38245, 459278, 160515, 310902, 236782,
            521023,  42288,  82548, 397568,  34739, 241805, 500298, 362601,
            187693, 139632, 141635, 200441, 107321,  72599, 359250,  66518,
            530654, 145245, 533011, 286590, 317323, 243648, 454060, 380886,
            307495, 392814, 537632, 288754, 524681, 503291,  51687,  28457,
            290536, 432115, 566544,  28232],
           dtype='int64', name='index')

In [10]:
# return the user's dataset given the sampled indices but with rows that have not been reviewed
df = dataset[user].loc[batch_indices].loc[~dataset[user].reviewed]
indices = df.index

# indices in batch that havent been reviewed
indices

Int64Index([ 34561, 250226, 328562, 260488, 567911,  69616, 544614, 574056,
            224878, 227250, 274951, 128062,  67388, 265521, 399504, 418206,
            371608, 420438, 580573, 445860, 576365, 305482, 366571, 418992,
            125671, 254945, 252887, 473485,  75026, 494942, 273298, 389623,
            428408, 234677, 186454,   9522, 317052, 474140, 489705,  27937,
             12256,  90785, 396561, 233878, 457919,  36048, 373581, 351670,
            562448, 432349, 296739, 122783, 438533,  38245, 459278, 160515,
            310902, 236782, 521023,  42288,  82548, 397568,  34739, 241805,
            500298, 362601, 187693, 139632, 141635, 200441, 107321,  72599,
            359250,  66518, 530654, 145245, 533011, 286590, 317323, 243648,
            454060, 380886, 307495, 392814, 537632, 288754, 524681, 503291,
             51687,  28457, 290536, 432115, 566544,  28232],
           dtype='int64', name='index')

In [12]:
# prepare data to review for user
i = 0  # starting position in the list of indices sampled  (Saved to df functioning as sentence idx)
prev = 0  # the stored number of sentences shown previous to the centre trigger sentence (Saved to df)
fwd = 0  # the stored number of sentences shown after the centre trigger sentence (saved to df)
event = df.loc[indices[i]]  # first event

def get_index_text(idx):
    return f"<h3>Current Index: {indices[i]} / {len(source)}</h3>"

def get_trigger_text(idx):  
    trigger_words = ''.join([f'<li>{w}</li>' for w in df.loc[idx,'triggers'].split(',')])
    return f"<h3>Trigger Words</h3><ul>{trigger_words}</ul>"

def yes_pressed(b):
    label_sentence(label=True)

def no_pressed(b):
    label_sentence(label=False)
    
def revert_pressed(b):
    revert_changes()

def label_sentence(label):
    global i

    # get confidence from button
    confidence = confidence_toggle.get_interact_value()
    
    # assign data
    df.loc[indices[i], 'reviewed'] = True
    df.loc[indices[i], 'label'] = label
    df.loc[indices[i], 'confidence'] = confidence

    # save lower_idx and upper_idx
    if save_chunk_toggle.get_interact_value():
        df.loc[indices[i], 'lower_bound'] = prev_input.value
        df.loc[indices[i], 'upper_bound'] = fwd_input.value
        
    # increment to next index
    i += 1
    
    event = df.loc[indices[i]]  # get index position, not loc
    idx_text.value = get_index_text(indices[i])
    confidence_toggle.value = 'High'
    triggers_widget.value = get_trigger_text(indices[i])
    prev_input.value = 0
    fwd_input.value = 0

    # sentence_text = files[event.filename][event.idx]
    with event_output:
        event_output.clear_output()
        
        display_ent([nlp(files[event.filename][event.idx])])
        
    with prev_output:
        prev_output.clear_output()
        prev_output.layout.display = "none"

    with fwd_output:
        fwd_output.clear_output()
        fwd_output.layout.display = "none"
        
def revert_changes():
    global i
    global prev
    global fwd
    
    # get data from toggle button
    confidence = confidence_toggle.get_interact_value()
    
    # revert idx back one
    i -= 1
    
    # reset data
    df.loc[indices[i], 'reviewed'] = False
    df.loc[indices[i], 'label'] = False
    df.loc[indices[i], 'confidence'] = None
    df.loc[indices[i], 'lower_bound'] = 0
    df.loc[indices[i], 'upper_bound'] = 0
    
    # get event
    event = df.loc[indices[i]]
    
    # update widget values
    idx_text.value = get_index_text(indices[i])
    confidence_toggle.value = 'High'
    triggers_widget.value = get_trigger_text(indices[i])
    prev_input.value = 0
    fwd_input.value = 0

    # sentence_text = files[event.filename][event.idx]
    with event_output:
        event_output.clear_output()
        
        display_ent([nlp(files[event.filename][event.idx])])
        
    with prev_output:
        prev_output.clear_output()
        prev_output.layout.display = "none"

    with fwd_output:
        fwd_output.clear_output()
        fwd_output.layout.display = "none"

def save_pressed(b):
    # print('Dataset saved.')
    dataset[user].loc[indices] = df
    dataset[user].to_csv(labelled_path, index=True, index_label='index')

idx_text = widgets.HTML(value=get_index_text(indices[i]))
triggers_widget = widgets.HTML(value=get_trigger_text(indices[i]))
line_break = widgets.HTML(value='\n')

prev_input = widgets.BoundedIntText(
    value=0,
    min=-10,
    max=0,
    step=1,
    description='Previous',
    disabled=False
)

fwd_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Next',
    disabled=False
)

def update_chunk(b):
    global i
    global prev
    global fwd
    
    event = df.loc[indices[i]]
    
    prev = prev_input.value
    fwd = fwd_input.value

    with prev_output:
        if prev < 0:
            prev_output.clear_output()
            prev_chunk = list(nlp.pipe(files[event.filename][event.idx + prev : event.idx]))
            display_ent(prev_chunk)
            prev_output.layout.display = "block"
        else:
            prev_output.layout.display = 'none'
        
    with fwd_output:
        if fwd > 0:
            fwd_output.clear_output()
            fwd_chunk = list(nlp.pipe(files[event.filename][event.idx + 1 : event.idx + 1 + fwd]))
            display_ent(fwd_chunk)
            fwd_output.layout.display = "block"
        else:        
            fwd_output.layout.display = 'none'

def toggle_ner(toggle):
    global idx
    global prev
    global fwd
    
    toggle = ner_toggle.get_interact_value()
    
    prev = prev_input.value
    fwd = fwd_input.value
    
    if prev < 0:
        with prev_output:
            prev_chunk = list(nlp.pipe(files[event.filename][event.idx + prev : event.idx]))
            if toggle:
                prev_output.clear_output()
                display_ent(prev_chunk)
            else:
                prev_chunk_text = ' '.join([token.text for token in prev_chunk]).strip()
                prev_output.clear_output()
                display(prev_chunk_text)
                
            prev_output.layout.display = "block"
            
    if fwd > 0:
        with fwd_output:
            fwd_chunk = list(nlp.pipe(files[event.filename][event.idx + 1 : event.idx + 1 + fwd]))
            if toggle:
                fwd_output.clear_output()
                display_ent(fwd_chunk)
            else:
                fwd_chunk_text = ' '.join([token.text for token in fwd_chunk]).strip()
                fwd_output.clear_output()
                display(fwd_chunk_text)
                
            fwd_output.layout.display = "block"
            
prev_output = widgets.Output()
event_output = widgets.Output()
fwd_output = widgets.Output()

prev_output.layout.display = "none"
fwd_output.layout.display = "none"

yes = widgets.Button(description='Near Miss Event', button_style='success')
yes.on_click(yes_pressed)

no = widgets.Button(description='Not Near Miss Event', button_style='danger')
no.on_click(no_pressed)

save = widgets.Button(description='Save DataFrame', button_style='primary')
save.on_click(save_pressed)

revert = widgets.Button(description='Revert Index', button_style='warning')
revert.on_click(revert_pressed)

update = widgets.Button(description='Update Text Range', button_style='')
update.on_click(update_chunk)

update_ner = widgets.Button(description='Update NER Display', button_style='')
update_ner.on_click(toggle_ner)

ner_toggle = widgets.Checkbox(
    value=True,
    description='External NER',
    disabled=False
)

confidence_toggle = widgets.ToggleButtons(
    options=['Low', 'Medium', 'High'],
    value='High',
    description='Confidence',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=[
        'An uncertain interpretation worth reviewing',
        'A reasonable interpretation of a near miss',
        'A certain interpretation of a near miss'],
)

save_chunk_toggle = widgets.Checkbox(
    value=True,
    description='Save Text Chunk Bounds',
    disabled=False,
    #button_style='info',
    #tooltip='Saves lower and upper bound indices on text chunks'
)

settings_text =  widgets.HTML(value="<h3>Text Chunk Settings</h3>")
labeller_text = widgets.HTML(value="<h3>Labeller</h3>")

#text_chunk_box = widgets.VBox([settings_text, save_chunk_toggle])

text_buttons = widgets.HBox([update, save_chunk_toggle])
ner_buttons = widgets.HBox([update_ner, ner_toggle])
label_buttons = widgets.HBox([yes, no])
function_buttons = widgets.HBox([save, revert])
buttons = widgets.VBox([label_buttons, function_buttons])

with event_output:
    display_ent(nlp(files[event.filename][event.idx]))
    
with prev_output:
    display(widgets.HTML(value='\n'))

with fwd_output:
    display(widgets.HTML(value=''))
                            
display(idx_text, triggers_widget, settings_text, prev_input, fwd_input, text_buttons, ner_buttons,
        labeller_text, buttons, line_break, confidence_toggle, prev_output, line_break, event_output, 
        line_break, fwd_output,)

HTML(value='<h3>Current Index: 34561 / 580746</h3>')

HTML(value='<h3>Trigger Words</h3><ul><li>mineralisation</li></ul>')

HTML(value='<h3>Text Chunk Settings</h3>')

BoundedIntText(value=0, description='Previous', max=0, min=-10)

BoundedIntText(value=0, description='Next', max=10)

...

Output(layout=Layout(display='none'))

HTML(value='\n')

Output()

HTML(value='\n')

Output(layout=Layout(display='none'))

In [15]:
# df is temp data store for your data - shown below
df.loc[df.reviewed]

,filename,anumber,sentence_count,report_type,idx,triggers,reviewed,label,confidence,lower_bound,upper_bound
index,,,,,,,,,,,
34561,a080552_gindalbie 2008 annual report_17729535....,80552,175,Annual,69,mineralisation,True,False,High,0,0


In [16]:
# dataset[user] is your main dataset, indices are the current indices being labelled
# note that df does not get saved unless you manually overwrite!
dataset[user].loc[df.loc[df.reviewed].index]

,filename,anumber,sentence_count,report_type,idx,triggers,reviewed,label,confidence,lower_bound,upper_bound
index,,,,,,,,,,,
34561,a080552_gindalbie 2008 annual report_17729535....,80552,175,Annual,69,mineralisation,False,False,NaN,0,0


In [17]:
# save backup
dataset[user].loc[indices] = df
dataset[user].to_csv(os.path.join('data','events',f'{user}_dataset_backup.csv'), index=True, index_label='index')